<a href="https://colab.research.google.com/github/abhirockraj/pytorch_nn/blob/main/Finacial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install yfinance

     |████████████████████████████████| 6.3 MB 8.0 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=3f766f815afecfa1aa7c94814093a940b0ce2a2406d07021adcc75eaafab8785
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F

In [4]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
adni = yf.Ticker("ADANIENT.NS")

In [6]:
adni.financials

,2021-03-31,2020-03-31,2019-03-31,2018-03-31
Research Development,None,None,None,None
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,1.38541e+10,1.36432e+10,5.5967e+09,6.587e+09
Minority Interest,1.75144e+10,1.26337e+10,3.8777e+09,7.7786e+09
Net Income,9.2264e+09,1.13817e+10,7.1714e+09,7.5725e+09
Selling General Administrative,6.78426e+10,6.45792e+10,5.08845e+10,3.89138e+10
Gross Profit,9.28931e+10,8.74223e+10,7.05725e+10,5.92558e+10
Ebit,1.98188e+10,1.85189e+10,1.59071e+10,1.38368e+10
Operating Income,1.98188e+10,1.85189e+10,1.59071e+10,1.38368e+10
Other Operating Expenses,-1.397e+08,-3.964e+08,-1.168e+08,-1.34e+08


In [7]:
hist = adni.history(period="5y")

In [8]:
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2016-10-10,36.526430,37.595235,35.858427,36.900509,9051818,0.0,0
2016-10-13,36.847074,37.247876,35.831708,36.018749,4018770,0.0,0
2016-10-14,36.232508,36.606591,35.858429,36.125629,3967572,0.0,0
2016-10-17,36.285953,36.847074,35.537788,36.018749,5609924,0.0,0
2016-10-18,36.312676,36.740198,36.179074,36.633316,2036798,0.0,0


In [27]:
df_x_close_out = hist[['Open','High','Low']]
df_y_close_out = hist['Close']

In [28]:
df_x_close_out.reset_index(drop=True, inplace=True)
df_y_close_out.reset_index(drop=True, inplace=True)

In [11]:
df_x_close_out.shape

(1233, 3)

In [29]:
df_x_close_out_val= df_x_close_out[1:247]
df_y_close_out_val = df_y_close_out[1:247]
df_x_close_out_train= df_x_close_out[247:]
df_y_close_out_train = df_y_close_out[247:]

In [13]:
df_x_close_out_train.shape

(986, 3)

Pytorch model creation

In [30]:
input_size =3
hidden_size=3
lr= 0.001
no_of_epoch=100
batch =10
num_layer=3

In [31]:
df_x_close_out_val =torch.Tensor(df_x_close_out_val.values)
df_y_close_out_val =torch.Tensor(df_y_close_out_val.values)
df_x_close_out_train= torch.Tensor(df_x_close_out_train.values)
df_y_close_out_train= torch.Tensor(df_y_close_out_train.values)

In [16]:
df_x_close_out_train.shape

torch.Size([986, 3])

In [25]:
train_loader = DataLoader(train_loader,batch_size=batch,shuffle=False)
val_loader = DataLoader(val_loader,batch_size=batch,shuffle=False)

In [26]:
c=0
for x,y in train_loader:
  c+=1
  x = torch.flatten(x,start_dim=1)
  print(x)
  if(c>1):
    break

tensor([[64.2009, 65.5440, 64.2009],
        [65.0874, 66.2962, 63.1802],
        [64.4695, 64.8993, 63.6637],
        [64.8993, 65.0874, 64.2009],
        [64.8187, 64.9799, 62.5892],
        [63.3145, 63.9054, 62.8847],
        [63.3951, 66.3768, 62.7772],
        [66.2156, 68.2303, 65.7321],
        [67.9079, 69.7346, 67.2632],
        [68.2303, 68.9556, 67.3707]])
tensor([[68.1497, 69.4122, 66.6454],
        [68.4989, 70.7822, 67.7736],
        [70.8628, 71.4000, 69.0361],
        [70.1912, 73.2804, 69.3585],
        [72.3671, 72.3671, 69.8689],
        [71.3463, 72.6894, 70.9702],
        [71.6149, 72.8506, 70.8359],
        [71.9910, 76.6650, 71.4000],
        [76.9337, 81.3659, 76.6382],
        [80.4526, 81.8226, 75.8592]])


In [32]:
train_loader= torch.utils.data.TensorDataset(df_x_close_out_train, df_y_close_out_train)
val_loader =torch.utils.data.TensorDataset(df_x_close_out_val,df_y_close_out_val)

In [64]:
x,y=train_loader[0]
print(x)

tensor([64.2009, 65.5440, 64.2009])


In [143]:
class NeuralNet(pl.LightningModule):
    def __init__(self):
         super(NeuralNet,self).__init__()
         self.model = nn.Sequential(
            nn.LSTM(input_size= 3,hidden_size= 50,batch_first=True),
            nn.LSTM(input_size=50,hidden_size= 50),
            nn.LSTM(input_size=50,hidden_size= 3),
            nn.Linear(3, 1))
    def forward(self,x):
         x = self.model(x)
         return x
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        return optimizer
    def training_step(self, train_batch, batch_idx):
        x,y = train_batch
        x= torch.reshape(x,(1,10,3))
        print(x)
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss)
        return loss
    def validation_step(self, val_batch, batch_idx):
        x,y = val_batch
        x= torch.reshape(x,(1,10,3))
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('val_loss', loss)
    
    def train_dataloader(self):
        return DataLoader(train_loader, batch_size=batch)

    def val_dataloader(self):
        return DataLoader(val_loader, batch_size=batch)    

In [144]:
trainer = Trainer(fast_dev_run=True)
model = NeuralNet()
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 32.1 K
-------------------------------------
32.1 K    Trainable params
0         Non-trainable params
32.1 K    Total params
0.128     Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:327: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: -1it [00:00, ?it/s]

tensor([[[64.2009, 65.5440, 64.2009],
         [65.0874, 66.2962, 63.1802],
         [64.4695, 64.8993, 63.6637],
         [64.8993, 65.0874, 64.2009],
         [64.8187, 64.9799, 62.5892],
         [63.3145, 63.9054, 62.8847],
         [63.3951, 66.3768, 62.7772],
         [66.2156, 68.2303, 65.7321],
         [67.9079, 69.7346, 67.2632],
         [68.2303, 68.9556, 67.3707]]])


AttributeError: ignored

In [86]:
a= a = torch.Tensor([[0,1,2],[3,4,5],[6,7,8],[6,7,8],[6,7,8],[6,7,8],[6,7,8],[6,7,8],[6,7,8],[6,7,8]])
# a.unsqueeze_(-1)
# a = a.expand(10,10,3)
a=torch.reshape(a,(1,10,3))
print(a)

tensor([[[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.],
         [6., 7., 8.],
         [6., 7., 8.],
         [6., 7., 8.],
         [6., 7., 8.],
         [6., 7., 8.],
         [6., 7., 8.],
         [6., 7., 8.]]])


In [76]:
a.reshape(-1,3)

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.],
        [6., 7., 8.],
        [6., 7., 8.],
        [6., 7., 8.],
        [6., 7., 8.],
        [6., 7., 8.],
        [6., 7., 8.],
        [6., 7., 8.]])

In [ ]:
a.shape

(2, 3)

In [ ]:
b= (2,1)

In [ ]:
x,y= b

In [ ]:
print(x)

2
